In [1]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

# New Section

In [18]:
!python -m spacy download en_core_web_lg

Traceback (most recent call last):
  File "c:\Users\Dhruv Thakkar\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connection.py", line 203, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Dhruv Thakkar\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    raise err
  File "c:\Users\Dhruv Thakkar\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\util\connection.py", line 73, in create_connection
    sock.connect(sa)
TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\Dhruv Thakkar\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3

In [10]:
import json
f1 = open('./Datasets/typeA_B_train_dataset.json')
f2 = open('./Datasets/typeA_B_test_dataset.json')

TRAIN_DATA = json.load(f1)
TEST_DATA = json.load(f2)


In [11]:
for text, annot in tqdm(TRAIN_DATA['annotations']):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start,end,label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)
db.to_disk("./Datasets/typeA_B_train_dataset.spacy")

100%|██████████| 92/92 [00:00<00:00, 7474.16it/s]


In [12]:
for text, annot in tqdm(TEST_DATA['annotations']):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start,end,label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)
db.to_disk("./Datasets/typeA_B_test_dataset.spacy")

100%|██████████| 15/15 [00:00<00:00, 2878.07it/s]

Skipping entity
Skipping entity


In [17]:
!python -m spacy train ./config.cfg --output ./model --paths.train ./Datasets/typeA_B_train_dataset.spacy --paths.dev ./Datasets/typeA_B_test_dataset.spacy

^C


ℹ Saving to output directory: model
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     69.17   19.39   14.80   28.14    0.19
 36     200          1.79    888.61   99.66   99.33  100.00    1.00
 80     400          0.00      0.00   99.66   99.33  100.00    1.00
131     600          0.00      0.00   99.66   99.33  100.00    1.00
198     800          0.00      0.00   99.66   99.33  100.00    1.00
⚠ Aborting and saving the final best model. Encountered exception:
MemoryError((549, 768), dtype('float32'))


Traceback (most recent call last):
  File "c:\Users\Dhruv Thakkar\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy\training\loop.py", line 135, in train
    raise e
  File "c:\Users\Dhruv Thakkar\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy\training\loop.py", line 118, in train
    for batch, info, is_best_checkpoint in training_step_iterator:
  File "c:\Users\Dhruv Thakkar\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy\training\loop.py", line 220, in train_while_improving
    nlp.update(
  File "c:\Users\Dhruv Thakkar\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy\language.py", line 1186, in update
    proc.update(examples, sgd=None, losses=losses, **component_cfg[name])  # type: ignore
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "spacy\pipeline\transition_parser.pyx", line 438, in spacy.pipeline.transition_parser.Parser.update
  File "spacy\ml\parser_model.pyx", line 333, in spacy

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [19]:
nlp_ner = spacy.load("./model/model-best")

In [ ]:
!ls /content/gdrive/MyDrive/Saved\ Models/typeA_model/

In [ ]:
nlp_ner.to_disk("/content/gdrive/MyDrive/Saved\ Models/typeA_model/")

In [20]:
nlp = spacy.load("en_core_web_sm")

# Input paragraph
#input_text = "I want to set cell A1 to the value 20, and then put 50 into cell B3."
#input_text = "Assign 'Inventory' to cell E9, fill cell C7 with 'Project Deadline', insert 500 at cell F2, and write 'Customer Names' in cell A3"
#input_text = "Please ensure that 'Product Inventory' is typed in cell G2, input 'Expense Report' into cell D9, and fill cell G1 with the 'Task List'."
input_text = "Integrate the Contact List into cell A3, append 60 to cell E4, affix 'Budget Summary' at cell C5, and introduce 10 at cell A8"
#input_text = "Delete A"


# Tokenize and process the input text
doc = nlp(input_text)

# Initialize a list to store sentences
sentences = []

# Keywords to split sentences
split_keywords = ["and", "or", "then", "but", ","]

current_sentence = ""

for token in doc:
    if token.text in split_keywords:
        if current_sentence:
            sentences.append(current_sentence.strip())
        current_sentence = ""
    else:
        current_sentence += " " + token.text

# Add the last sentence
if current_sentence:
    sentences.append(current_sentence.strip())

# Now 'sentences' is a list of separated sentences that you can use with your model
# Print the separated sentences
#for i, sentence in enumerate(sentences, start=1):
    #print(f"Sentence {i}: {sentence}")

print(sentences)

['Integrate the Contact List into cell A3', 'append 60 to cell E4', "affix ' Budget Summary ' at cell C5", 'introduce 10 at cell A8']


In [6]:
def concat_instruction(type, cells, values):
  for t in type:
    print(t)
    if t=="TYPE A":
      # the sentence is of INSERT

      cmd = "Insert " + str(cells[0]) + " "+ str(values[0])
      print(cmd)
    elif t=="TYPE B":
      # the sentence is of type delete
      cmd = "Delete " + str(cells[0])
      print(cmd)


In [21]:
# Assuming 'nlp_ner' is custom-trained spaCy NER model

for sentence in sentences:
  print(sentence)
  values = []
  cells = []
  type =[]

  doc = nlp_ner(sentence)
  for ent in doc.ents:
    if ent.label_ == "VALUE":
        values.append(ent.text)
    elif (ent.label_ == "TYPE A" or ent.label_ == "TYPE B" or ent.label_ == "TYPE C"):
        type.append(ent.label_)
    elif ent.label_ == "CELL":
        cells.append(ent.text)
  concat_instruction(type, cells, values)

Integrate the Contact List into cell A3
TYPE A
Insert A3 Contact List
append 60 to cell E4
TYPE A
Insert E4 60
affix ' Budget Summary ' at cell C5
TYPE A
Insert C5 ' Budget
introduce 10 at cell A8
TYPE A
Insert A8 10
